In [2]:
import copy, numpy as np

data = open('HP1.txt','r', encoding="utf8").read();
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print(data_size,", ",vocab_size)

431677 ,  79


In [3]:
# Dictionary of input chars & indices
char_to_ix = {ch:i for i, ch in enumerate(chars)}
ix_to_char = {i:ch for i,ch in enumerate(chars)}
print(char_to_ix)
print(ix_to_char)

{'K': 0, 'V': 1, 'l': 2, 'p': 3, ')': 4, 'g': 5, '2': 6, 't': 7, 'r': 8, 'v': 9, 's': 10, 'A': 11, 'S': 12, 'B': 13, 'm': 14, '-': 15, '"': 16, '\\': 17, 'Q': 18, 'W': 19, ':': 20, '9': 21, 'e': 22, 'k': 23, 'u': 24, 'J': 25, 'y': 26, 'Z': 27, 'f': 28, '3': 29, "'": 30, 'w': 31, 'N': 32, '*': 33, '8': 34, 'F': 35, 'z': 36, '.': 37, '~': 38, 'G': 39, 'j': 40, 'c': 41, 'b': 42, 'U': 43, 'L': 44, 'X': 45, 'D': 46, 'a': 47, 'h': 48, 'T': 49, 'R': 50, 'o': 51, '?': 52, '6': 53, 'I': 54, 'O': 55, 'q': 56, 'H': 57, 'n': 58, '\n': 59, 'P': 60, ' ': 61, 'C': 62, ';': 63, '4': 64, 'x': 65, 'M': 66, 'E': 67, 'd': 68, '(': 69, '1': 70, '!': 71, 'i': 72, '0': 73, '5': 74, '\t': 75, 'Y': 76, ',': 77, '7': 78}
{0: 'K', 1: 'V', 2: 'l', 3: 'p', 4: ')', 5: 'g', 6: '2', 7: 't', 8: 'r', 9: 'v', 10: 's', 11: 'A', 12: 'S', 13: 'B', 14: 'm', 15: '-', 16: '"', 17: '\\', 18: 'Q', 19: 'W', 20: ':', 21: '9', 22: 'e', 23: 'k', 24: 'u', 25: 'J', 26: 'y', 27: 'Z', 28: 'f', 29: '3', 30: "'", 31: 'w', 32: 'N', 33: '*

In [4]:
# demo of onehot encoding
vector_for_char_a = np.zeros((vocab_size,1))
vector_for_char_a[char_to_ix['a']] = 1
print(vector_for_char_a.ravel())

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0.]


In [5]:
# Hyperparams
N = 2;
hidden_size = 100
seq_length = 25
learning_rate = 1e-1
# the lower the learning rate, the quicker the network abandons old belief for new input
# e.g. train images on dogs, give a cat, low learning rate will consider cat is anormally rather than dog

# Model params
# Input to first hidden
Wxh1 = np.random.randn(hidden_size,  vocab_size)* 0.01 # input to hidden (input is onehot encoded)
Bxh1 = np.zeros((hidden_size,1))

# Hiddens
Whihj = np.random.randn(N-1,hidden_size, hidden_size)* 0.01 # hidden_i to hidden_i+1
Whihi = np.random.randn(N,  hidden_size, hidden_size)* 0.01 # recurrent hidden .

Bhihj = np.zeros((N-1,hidden_size,1))

# last hidden to Output 
Whjy = np.random.randn(vocab_size,  hidden_size)* 0.01 # hidden to output(decode the output)
Bhjy = np.zeros((vocab_size,1))


In [6]:
# softmax helper
def softmax(seq):
    return np.exp(seq)/ np.sum(np.exp(seq))

def softmax_array(two_D_seq,t):
    return np.exp(two_D_seq[t])/ np.sum(np.exp(two_D_seq[t]))

In [11]:
# Loss function - training
def lossFunction(inputs, targets,N, prev_hiddens):
    # p is softmax probability    
    xs, his, ys, ps = {},{},{},{};    
    for i in range(N):
        his[i] = {};
        his[i][-1] = copy.deepcopy(prev_hiddens[i])  
    loss = 0;
    
    # Fwd pass    
    for t in range(len(inputs)):
        # One hot encoding for the input char using our dictionary
        xs[t] = np.zeros((vocab_size,1));
        xs[t][inputs[t]] = 1; 
        
        his[0][t] = np.tanh(np.dot(Wxh1,xs[t]) + np.dot(Whihi[0],his[0][t-1]) + Bxh1);      
        for i in range(1,N):
            his[i][t] = np.tanh(np.dot(Whihj[i-1],his[i-1][t]) + np.dot(Whihi[i],his[i][t-1]) + Bhihj[i-1]);
        
        ys[t] = np.dot(Whjy,his[N-1][t]) + Bhjy;
        ps[t] = softmax(ys[t]);
        char_idx = targets[t]
        loss += -np.log(ps[t][char_idx,0]) 
        # ps[t][targets[t]] is the prob. node corrs. to. t_th char in the label array

        
    # Gradient value holders
    dWxh1, dWhihj, dWhjy = np.zeros_like(Wxh1),np.zeros_like(Whihj),np.zeros_like(Whjy)
    dWhihi               = np.zeros_like(Whihi)
    dBxh1, dBhihj, dBhjy = np.zeros_like(Bxh1),np.zeros_like(Bhihj),np.zeros_like(Bhjy)
    dhinext = np.zeros_like(prev_hiddens)
    
    # Bwd pass
    for t in reversed(range(len(inputs))):
        
        # prob. p to output y
        dy = copy.deepcopy(ps[t])
        dy[targets[t]] -= 1 # this is how we calculate loss using onehot encoding
        
        # y to hj 
        dWhjy += np.dot(dy, his[N-1][t].T);        
        dBhjy += dy # derivative w.r.t bias is 1        
        dh_curr = np.dot(Whjy.T,dy) + dhinext[N-1] # back prop the error from y into h2
                
        # hj to hj-1 and hj_prev
        # e.g. if N = 2, it needs to itr 1 time
        # and j will start at 1
        for j in reversed(range(1,N)):
            dhjraw = (1-his[j][t]*his[j][t])*dh_curr # back prop thru tanh
            dBhihj[j-1] += dhjraw  # derivative of Wx+b w.r.t b is 1; d_loss/d_b = d_loss/d_H * d_H/d_b
            dWhihj[j-1] += np.dot(dhjraw, his[j-1][t  ].T) # derivative of Wx+b w.r.t W is x
            dWhihi[j  ] += np.dot(dhjraw, his[j  ][t-1].T)
            dhinext[j] = np.dot(Whihi[j].T, dhjraw)  
            
            dh_curr = np.dot(Whihj[j-1].T,dh_curr) + dhinext[j-1] # back prop the error from h_j into h_j-1
        
        # h1 to x and h1_prev
        dhjraw = (1-his[0][t]*his[0][t])*dh_curr # back prop thru tanh
        dBxh1     += dhjraw  
        dWxh1     += np.dot(dhjraw,    xs[t  ].T) 
        dWhihi[0] += np.dot(dhjraw,his[0][t-1].T)
        dhinext[0] = np.dot(Whihi[0].T, dhjraw)  

    # Can be replaced using LSTM structure
    for dparam in [dWxh1, dWhjy, dBxh1, dBhjy]:
        np.clip(dparam,-5,5,out=dparam) # mitigate gradient vanish
    
    for j in range(N-1):
        for dparam in [dWhihj[j], dWhihi[j], dBhihj[j]]:
            np.clip(dparam,-5,5,out=dparam) # mitigate gradient vanish
            
    np.clip(dWhihi[N-1],-5,5,out=dWhihi[N-1]) # mitigate gradient vanish         
    
    h_ret = np.zeros((N,hidden_size,1))
    for j in range(N):
        h_ret[j] = his[j][len(inputs)-1]
        
        
    return loss,dWxh1,dWhihj,dWhihi,dWhjy, dBxh1,dBhihj, dBhjy, h_ret;


 

In [12]:
# prediction
def sample(h,hN,seed_ix,n):
    x = np.zeros((vocab_size,1))
    # one hot encode
    x[seed_ix] = 1;
    ixes = [] # empty sentence

    for t in range(n):

        h[0] = np.tanh(np.dot(Wxh1 ,x ) + np.dot(Whihi[0],h[0]) + Bxh1 );
        for i in range(1,hN):
            h[i] = np.tanh(np.dot(Whihj[i-1],h[i-1]) + np.dot(Whihi[i],h[i]) + Bhihj[i-1]);
        y = np.dot(Whjy,h[hN-1]) + Bhjy;
        p = softmax(y);

        # sample the output
        ix = np.random.choice(range(vocab_size), p=p.ravel())
        
        # encode this output
        x = np.zeros((vocab_size,1))
        x[ix] = 1;

        ixes.append(ix)
        # if n > 1, it will predict more than 1 subsequent chars
        
    txt = ''.join(ix_to_char[ix] for ix in ixes)
    print ("----\n %s \n----" % (txt,))
    
# test
hprev = np.zeros((N,hidden_size,1)) # reset RNN memory  
#predict the 200 next characters given 'a'
sample(hprev,N,char_to_ix['a'],200)

----
  esbka.eiei shshhtefekso"asihMltare ss goC osl ngrbetyb hwnht ldrcrsehd er npfoeingn utidetoo riavn uunahenilcitieeueehwksrnOko  la hethhg, htme e w.e niick ywg . tuemelmesp hkje'dirdhwhisst ehi tynl  
----


In [13]:
# Training using Adagrad (decreasing learning rate)
position = 0
inputs = [char_to_ix[ch] for ch in data[position:position+seq_length]]
print(data[position:position+seq_length])
print("inputs",inputs)
targets = [char_to_ix[ch] for ch in data[position+1:position+seq_length+1]] 
print(data[position+1:position+seq_length+1])
print("targets",targets)



Harry Potter and the Sorc
inputs [57, 47, 8, 8, 26, 61, 60, 51, 7, 7, 22, 8, 61, 47, 58, 68, 61, 7, 48, 22, 61, 12, 51, 8, 41]
arry Potter and the Sorce
targets [47, 8, 8, 26, 61, 60, 51, 7, 7, 22, 8, 61, 47, 58, 68, 61, 7, 48, 22, 61, 12, 51, 8, 41, 22]


In [14]:
# Real training

n, position = 0,0

# for Adaptive Gradient descent
mWxh1 = np.zeros_like(Wxh1);
mWhihi = np.zeros_like(Whihi);
mWhihj = np.zeros_like(Whihj);
mWhjy = np.zeros_like(Whjy);

mBxh1 = np.zeros_like(Bxh1);
mBhihj = np.zeros_like(Bhihj);
mBhjy = np.zeros_like(Bhjy);

smooth_loss = -np.log(1.0/vocab_size)*seq_length;

epoch = 200*1000;
sample_length = 200;
hprev = np.zeros((N,hidden_size,1))

while n<epoch:

    if(position+seq_length+1 >= len(data) or n == 0):
        
        hprev = np.zeros_like(hprev)
        position = 0;
        
    inputs = [char_to_ix[ch] for ch in data[position:position+seq_length]]
    targets = [char_to_ix[ch] for ch in data[position+1:position+seq_length+1]] 
    
    loss,dWxh1,dWhihj,dWhihi,dWhjy, dBxh1,dBhihj, dBhjy, hprev = lossFunction(inputs,targets,N,hprev)  
    smooth_loss = smooth_loss*0.999+loss*0.001
    
    if(n%1000 == 0):
        print(n,"loss",smooth_loss)
        sample(hprev,N,inputs[0],sample_length);
        
    # update
    for param, dparam, mem in zip([Wxh1,Bxh1,Whjy,Bhjy],
                                  [dWxh1,dBxh1,dWhjy,dBhjy],
                                  [mWxh1,mBxh1,mWhjy,mBhjy]):
        
        mem += dparam*dparam;
        param += -learning_rate * dparam *  1 / np.sqrt(mem +1e-8) # Adagrad
    
    for i in range(N-1):
        for param, dparam, mem in zip([ Whihj[i], Whihi[i], Bhihj[i]],
                                      [dWhihj[i],dWhihi[i],dBhihj[i]],
                                      [mWhihj[i],mWhihi[i],mBhihj[i]]):
            mem += dparam*dparam;
            param += -learning_rate * dparam *  1 / np.sqrt(mem +1e-8) # Adagrad    
    
    mWhihi[N-1] += dWhihi[N-1]*dWhihi[N-1];
    Whihi[N-1] += -learning_rate * dWhihi[N-1] *  1 / np.sqrt(mWhihi[N-1] +1e-8) # Adagrad    
        
    position += seq_length;
    n += 1;
    
        

0 loss 109.20401781297652
----
 eav ornhws peyna h eciH omwba rytlcccdc lcoonyrShigueotnsly
 hgwslheo istn c ns w  i  esyien nelgeednWtma'roat oaaittn.wagn hde  r nila t delewt stpso.e e'mtero:n od
fttla iaabh mtrviml as- adeTeoeyoe 
----
1000 loss 93.87083013834888
----
 Mu,ttol Di;ge wrtDoShoia htirdhotrotk  wfe we dvPelpg o sei  g l h  fmbawFet Gyytdlte,ns gaar usgtl ogee,getndcaeo u areEcwef hwhe aoue fpitHtVt sdunewesl uer\W ate iehnler  ezntlmi ,torio  gdyloav,dr 
----
2000 loss 82.83375848778253
----
 hoesa ue?vebs jiyf t'aipoene aos eed hrinlgb,l teee thrnhrrid,es, nd he inera" rh hHnorg h"s,drHirv hl oh VO' tBrlmeer nhd her ant stt HAneVe hud tus eihi br. lwp Hs. aen ph gunfrs ictDllsaasir yust E 
----
3000 loss 77.80788808127976
----
 l hii. wi lrp. hinh. coe -irb
sdet,ir,E iowny"sikmr - gf'ne, oDmard "oYetiParl ton.!. wk aous tun todhyl" 7hvl. Putetd Wuplnoud yisedrr mnlfindei nnmR aaaeetleut,s vow de iu pmG,im" meone spo, shnr UD 
----
4000 loss 75.13541300062374
----
 uard

KeyboardInterrupt: 

In [43]:
print(n)

100000
